In [ ]:
!sudo apt-get install libmagickwand-dev
!pip install --no-cache-dir \
    opencv-python-headless==4.6.*\
    rawpy==0.17.* \
    pandas \
    Pillow==7.1.2 \
    scikit-image==0.16.2 \
    scipy \
    tqdm \
    Wand

In [3]:
!git clone https://github.com/andreacos/BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation

fatal: destination path 'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation' already exists and is not an empty directory.


Scripts from repo:  
dataset.py creates a training and test dataset of double JPEG compressed image patches for a given set of JPEG quality factor pairs (QF1, QF2).  
redict_2terms.py to test the model, make sure to adapt the script to point to the right to-be-tested model file.  
usage.py contains an example of how to test the provided pre-trained models on a single image.  
utils.py functions for manage QF, labels, dct coeffs

In [4]:
import io
import os
import sys
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
from glob import glob
from PIL import Image

In [5]:
sys.path.insert(1, "BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation")

In [6]:
# Импорт методов из репо BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation
from networks import custom_two_terms_loss_wrapper, custom_softmax_activation, custom_mse_wrapper
from utils import max_min_coefficient, label2coefficient, string2Q

In [7]:
# Загрузка предобученной модели
model_path = 'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation/models/model_QF1_60-98_QF2_90-2-term-loss.h5'
# Max value for coefficients
max_coeffs, _ = max_min_coefficient(quality_range=(50, 100),
                                    n_coeffs=15,
                                    zig_zag_order=True)

model = tf.keras.models.load_model(model_path,
                                    custom_objects=({'custom_softmax': custom_softmax_activation(max_coeffs),
                                                    'custom_two_terms_loss_wrapper': custom_two_terms_loss_wrapper(max_coeffs, 0.8),
                                                    'custom_mse': custom_mse_wrapper(max_coeffs)}))

# Создание массива векторов ДКП-коээфициентов для соответсвующих QF
# Load the table linking each pair of JPEG quality factors to the corresponding Q's coefficients
qf_map = np.load('BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation/resources/qf1_qf2_map_90.npy', allow_pickle=True)
# Значения QF [:, [QF1, QF2]]
QFs = qf_map[:,:2]
# Массив векторов коэффициентов
dct_coeffs = []
for rec in qf_map:
  #  Converts a comma separated string to vector of dct coeffs
  dct_coeffs.append(string2Q(rec[2], size=(8, 8), flatten=True)[:15])
dct_coeffs = np.array(dct_coeffs)

In [9]:
# Сжатие изображения
# Принимает и возвращает картинку как numpy array
def compress_image(img, qfs):
  for qf in qfs:
    # Compress image into buffer
    jpeg_encoded = cv2.imencode('.jpg', img, [int(cv2.IMWRITE_JPEG_QUALITY), qf])[1]
    # Back to numpy rgb image
    jpeg_encoded_image = Image.open(io.BytesIO(jpeg_encoded))
    img = np.array(jpeg_encoded_image)
  return img

# Расчёт weighted MSE
def mse_weighted(pred, dct_coeffs):
  evals = []
  for vector in dct_coeffs:
    weights = np.log(vector + 1)+1
    diff = (pred - vector)**2
    weighted_mse = np.sum(diff*weights) / (np.sum(weights)*len(weights))
    evals.append(weighted_mse)
  return np.argmin(evals)

# Расчёт MSE
def mse(pred, dct_coeffs):
  evals = np.mean((dct_coeffs - pred) **2, axis=1)
  return np.argmin(evals)

# Подготовка изображения для теста from usage.py
# Принимает и возвращает картинку как numpy array
def preprocess_input(im, target_size, scale=255.):
  if im.shape != target_size:
    im = cv2.resize(im, target_size)

  # Переводим картинку в цветовое пространство YCbCr
  # Берем только канал Y (Luminance)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2YCrCb)[:,:,0]

  # Нормализация
  return im.astype(np.float32) / scale

# Предсказание QF
def make_prediction(input, dct_coeffs, max_coeffs):
  prediction = model.predict(np.expand_dims(input, [0, -1]), verbose=0)
  predicted_label = label2coefficient(prediction.flatten(), max_coefficients=max_coeffs)
  
  w_mse_idx = mse_weighted(predicted_label, dct_coeffs)
  mse_idx = mse(predicted_label, dct_coeffs)
  return QFs[w_mse_idx], QFs[mse_idx]

In [14]:
# Проверка работы модели
img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/data/source_images/1.tif')
tmp = compress_image(img, (60, 90))
tmp = tmp[:64,:64]
tmp = preprocess_input(tmp, (64, 64), 255.)
prediction = model.predict(np.expand_dims(tmp, [0, -1]), verbose=0)
predicted_label = label2coefficient(prediction.flatten(), max_coefficients=max_coeffs)
print("Predicted label:", predicted_label)

w_mse_idx = mse_weighted(predicted_label, dct_coeffs)
mse_idx = mse(predicted_label, dct_coeffs)
print('QF if WMSE: ', QFs[w_mse_idx])
print('QF if MSE:', QFs[mse_idx])
#pred_w, pred = make_prediction(tmp, dct_coeffs, max_coeffs)

Predicted label: [12  9  9 11  9  8 12 11 10 11 14 13 12 15 19]
QF if WMSE:  [61 90]
QF if MSE: [61 90]


In [ ]:
# Программно сгенерировать 5 принципиально различных ситуаций: 
# однократное сжатие, 𝑄𝐹1 < 𝑄𝐹2, 𝑄𝐹1 ≪ 𝑄𝐹2, 𝑄𝐹1 > 𝑄𝐹2, 𝑄𝐹1 ≈ 𝑄𝐹2. 
experiment_qf = [
    (60,90),
    (85,90),
    (89,90),
    (90,90),
    (98,90),
]

experiment_data = []
img_path = '/content/drive/MyDrive/Colab Notebooks/data/source_images/'
for e in experiment_qf:
  for path in glob(img_path+'*.tif'):
    img = cv2.imread(path)
    tmp = compress_image(img, e)
    tmp = tmp[:64,:64]
    tmp = preprocess_input(tmp, (64, 64), 255.)
    pred_w, pred = make_prediction(tmp, dct_coeffs, max_coeffs)
    experiment_data.append([e[0], pred_w[0], np.abs(pred_w[0]-e[0]), pred[0], np.abs(pred[0]-e[0])])

print('Experiment data sample: ', experiment_data[0])

Experiment data sample:  [60, 61, 1, 61, 1]


In [ ]:
res_df = pd.DataFrame(experiment_data, columns=['QF1', 'Prediction_WMSE', 'Error_WMSE', 'Prediction', 'Error'])
res_m_df = res_df.groupby('QF1').mean()

In [ ]:
res_df.head()

,QF1,Prediction_WMSE,Error_WMSE,Prediction,Error
0,60,61,1,61,1
1,60,62,2,62,2
2,60,61,1,61,1
3,60,63,3,63,3
4,60,66,6,66,6


In [ ]:
res_m_df

,Prediction_WMSE,Error_WMSE,Prediction,Error
QF1,,,,
60,62.500000,2.500000,62.500000,2.500000
85,83.833333,1.166667,83.833333,1.166667
89,92.750000,3.750000,92.750000,3.750000
90,95.083333,5.083333,95.000000,5.000000
98,95.666667,2.333333,95.583333,2.416667
